In [16]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
#Testing

In [3]:
df_test = pd.read_csv("/kaggle/input/test-data-sar-proj/test_data.csv")

In [4]:
df_test.shape

(5724, 3)

In [5]:
df_test.head()

,ID,article_link,text
0,27927,https://www.huffingtonpost.com/entry/teacher-e...,states slow to shut down weak teacher educatio...
1,1660,https://www.theonion.com/drone-places-fresh-ki...,drone places fresh kill on steps of white house
2,96,https://www.theonion.com/report-majority-of-in...,report: majority of instances of people gettin...
3,6237,https://local.theonion.com/sole-remaining-lung...,"sole remaining lung filled with rich, satisfyi..."
4,6650,https://www.huffingtonpost.com/entry/the-gops-...,the gop's stockholm syndrome


In [6]:
# ids_df = df_test['ID'].tolist()

In [7]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [8]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

In [9]:
model = BERTClass()

In [12]:
model.load_state_dict(torch.load("/kaggle/input/nlp-bert/bert_model_50epochs_70per.pth"))  
model.to(device)  

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [14]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe['text']
        self.ids = dataframe['ID']
#         self.targets = self.data["IS_SARCASTIC"]
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        ID = self.ids[index]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'ID': torch.tensor(ID, dtype=torch.long)
        }

In [17]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [18]:
# Creating the dataset and dataloader for the neural network

# train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=df_test

print("TEST Dataset: {}".format(test_dataset.shape))

testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

TEST Dataset: (5724, 3)


In [19]:
TEST_BATCH_SIZE = 1

In [29]:
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(testing_set, **test_params)

In [30]:
result_dict = {}
result_dict['ID']= []
result_dict["label"] = []

In [31]:
def validation():
    model.eval()
    fin_outputs=[]
    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ID = data['ID'].to(device, dtype = torch.long)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            val = torch.sigmoid(outputs).item()
            result_dict['ID'].append(ID.item())
            if val>=0.5:
                result_dict['label'].append(1)
            else:
                result_dict['label'].append(0)
#             fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return 

In [32]:
validation()

In [33]:
len(result_dict['ID'])

5724

In [34]:
len(result_dict['label'])

5724

In [35]:
df = pd.DataFrame(result_dict)
csv_file = "submission.csv"
df.to_csv(csv_file, index=False)